<a href="https://colab.research.google.com/github/dpicca/colab_projects/blob/main/youtube_tell_me_about_yourself_interview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube-tts-data-generator
!pip install youtube-search

In [ ]:
from youtube_search import YoutubeSearch

results = YoutubeSearch('tell me about yourself interview', max_results=10).to_json()
type(results)

str

In [ ]:
import json
f = open("links.txt", "a")
videos=json.loads(results)
for video in videos['videos']:
  f.write(f"https://www.youtube.com{video['url_suffix']}\n")
f.close()

In [ ]:
from youtube_tts_data_generator import YTSpeechDataGenerator

# First create a YTSpeechDataGenerator instance:

generator = YTSpeechDataGenerator(dataset_name='tmay')

# Now create a '.txt' file that contains a list of YouTube videos that contains speeches.
# NOTE - Make sure you choose videos with subtitles.

generator.prepare_dataset('links.txt')
# The above will take care about creating your dataset, creating a metadata file and trimming silence from the audios.



/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, defaultdict


Creating directory 'tmay_prep'..
Creating directory 'tmay_prep/downloaded'
Creating directory 'tmay_prep/split'
Creating directory 'tmay_prep/concatenated'
Creating directory 'tmay'..
Creating directory 'tmay/wavs'
Creating directory 'tmay/txts'
Completed downloading tmay1.wav from https://www.youtube.com/watch?v=kayOhGRcNt4
Completed downloading tmay2.wav from https://www.youtube.com/watch?v=5v-wyR5emRw
Completed downloading tmay3.wav from https://www.youtube.com/watch?v=EzGH3hZuJVk


/usr/local/lib/python3.7/dist-packages/youtube_tts_data_generator/youtube_tts_data_generator.py:346: NoSubtitleWarning: WARNING - video https://www.youtube.com/watch?v=cpm0aB9M-1g does not have subtitles. Skipping..
  NoSubtitleWarning,


Completed downloading tmay5.wav from https://www.youtube.com/watch?v=1ey58zDpBgE
Completed downloading tmay6.wav from https://www.youtube.com/watch?v=TQHW7gGjrCQ
Completed downloading tmay7.wav from https://www.youtube.com/watch?v=HMQlA-TIAsk
Completed downloading tmay8.wav from https://www.youtube.com/watch?v=2jkAj99sEzs
Completed downloading tmay9.wav from https://www.youtube.com/watch?v=MmFuWmzeiDs


/usr/local/lib/python3.7/dist-packages/youtube_tts_data_generator/youtube_tts_data_generator.py:346: NoSubtitleWarning: WARNING - video https://www.youtube.com/watch?v=Xdv4fSPCWyY does not have subtitles. Skipping..
  NoSubtitleWarning,
Processing tmay1.wav:   0%|          | 0/8 [00:00<?, ?it/s]

...

Completed verifying audios and their transcriptions in '/content/tmay_prep/split'.
You can find files data in '/content/tmay_prep/split/split.csv'
Reading audio data from 'split.csv'.
Processing audios shorter than 7 seconds..


100%|██████████| 844/844 [00:05<00:00, 155.91it/s]
0it [00:00, ?it/s]
  0%|          | 0/844 [00:00<?, ?it/s]

Processing audios longer than 7 seconds..
Completed concatenating audios and their transcriptions in '/content/tmay_prep/concatenated'.
Trimming silence from audios in '/content/tmay_prep/concatenated'.


100%|██████████| 844/844 [03:50<00:00,  3.66it/s]

Dataset 'tmay' has been generated. Wav files are placed in '/content/tmay/wavs'. Transcription files are placed in '/content/tmay/txts'.
Metadata is placed in '/content/tmay' as 'metadata.csv'.
Collected 0.19hours (679 seconds) of audio.


In [ ]:
!zip -r /content/tmay.zip /content/tmay
!zip -r /content/tmay_prep.zip /content/tmay_prep/

In [ ]:

from google.colab import files
files.download("/content/tmay_prep.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from adaptnlp.model_hub import HFModelHub

hub = HFModelHub()
models = hub.search_model_by_task('text2text-generation'); models

[Model Name: t5-11b, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-3b, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-base, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-large, Tasks: [summarization, text2text-generation, translation],
 Model Name: t5-small, Tasks: [summarization, text2text-generation, translation]]

In [ ]:
from adaptnlp import EasyTextGenerator
text = "China and the U.S. will begin to"
generator = EasyTextGenerator()
generated_text = generator.generate(text, model_name_or_path="mrm8488/t5-base-finetuned-emotion",max_length=2)

print(generated_text)

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

def get_emotion(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)

  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label

get_emotion("i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute") # Output: 'joy'



SyntaxError: ignored

In [ ]:
import logging

from simpletransformers.language_modeling import LanguageModelingModel

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "block_size": 512,
    "max_seq_length": 512,
    "learning_rate": 5e-6,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 8,
    "num_train_epochs": 3,
    "mlm": False,
    "output_dir": f"outputs/fine-tuned/",
}

model = LanguageModelingModel("gpt-neo", 'EleutherAI/gpt-neo-2.7B', args=train_args)

model.train_model(train_df)

#model.eval_model("data/test.txt")


KeyError: ignored